In [1]:
import os
import json
from tqdm import tqdm
import pandas as pd
import numpy as np

#### This was done for the old data. Please update the fields according to the new one based on the documentation

In [1]:
''' Recursive function to make sure all the fields are cleaned up.
    Make sure the values of keys are of similar types when using Elasticsearch
    Bulk Import.'''
def iterate_val(dictionary):
    if isinstance (dictionary,dict):
        for item in dictionary:
#             print(item)
            if item == 'tables':
                dictionary[item] = str(dictionary[item])
            if item == 'paras' and dictionary[item] == '0':
#                 print(dictionary[item])
                dictionary[item] = [{"Null Value Here":'0'}]
#                 print(dictionary[item])
            if item == '' or item == [] or item == {}:
                dictionary["Null Value Found"] = dictionary[item]
                del dictionary[item]
                continue
            if dictionary[item] == {Ellipsis}:
                dictionary[item] = {"Null Value Here" : "0"}
            if isinstance(dictionary[item],dict):
                iterate_val(dictionary[item])
            if isinstance(dictionary[item],list):
                for idx in dictionary[item]:
                    if isinstance(idx,dict):
                        iterate_val(idx)
    else:
        print("Check")
        print(dictionary)
    return dictionary

In [3]:
'''Path to the documents. (This was done for the old data)'''
path_to_parent = ''
dictionary = []
idx = 0
new_lst = []
for act in tqdm(os.listdir(os.path.join(path_to_parent))):
    if act.endswith('.json'):
        idx += 1
        x = ''
        x = str(os.path.join(path_to_parent, act))
        with open(x, 'r') as j:
            contents = json.loads(j.read())
            new_cont = iterate_val(contents[0])
                # for key_two in contents['Chapter']
            dictionary.append(new_cont)

100%|█████████████████████████████████████████████████████████████████████████████| 2450/2450 [00:02<00:00, 895.45it/s]


In [50]:
new_lst = []
print(len(dictionary))
for i in range(0,len(dictionary)):
    index = {"index":{"_index":"acts","_id":str(i)}}
    new_lst.append(index)
    new_lst.append(dictionary[i])
len(new_lst)

1351


2702

In [65]:
lst = [new_lst[i:i+50] for i in range(0, len(new_lst), 50)]

In [67]:
#This is to index the Acts at one go
import requests
for item in lst:
    with open('output.json', 'w') as f:
        for d in item:
            string = json.dumps(d)
            f.write(string+'\n')
    headers = {
    'Content-Type': 'application/json',}
    data = open('output.json', 'rb').read()
    response = requests.post('https://search-lawnics-h6i3e4ys7i26faaqoqsmqoew3m.us-east-2.es.amazonaws.com//_bulk', 
                             headers=headers, data=data, auth=('administrator', 'L@wnics2020'))

In [5]:
#This is to index acts with each section as a separate document
import requests
headers = {
'Content-Type': 'application/json',}
data = open('output_separate.json', 'rb').read()
response = requests.post('https://search-lawnics-h6i3e4ys7i26faaqoqsmqoew3m.us-east-2.es.amazonaws.com//_bulk', 
                         headers=headers, data=data, auth=('administrator', 'L@wnics2020'))